# Imports and data loading

In [1]:
import warnings

warnings.filterwarnings("ignore")
import scanpy as sc
import pertpy as pt

Global seed set to 0
/mnt/cip/home/p/pohll/mambaforge/envs/xie_env/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
/mnt/cip/home/p/pohll/mambaforge/envs/xie_env/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.


To use sccoda or tasccoda please install ete3 with pip install ete3

In [3]:
adata = sc.read("/home/p/pohll/Desktop/MP/data/galapagos_bleo_new_dr_leiden.h5ad")